# Biopython. Conexión a bases de datos externas

El NCBI no es la única base de datos bioinformáticos, existen varias más que son de interés. La mayoría de las bases de datos bioinformáticas ofrecen formas de conectarse a ellas de manera programática, esto es, en lugar de hacerlo desde un sitio web en forma interactiva, la conexión es directa desde un script en nuestra computadora a uno de los servidores de estas bases de datos. Biopython cuenta con objetos y métodos para conectarse a diferentes bases de datos. Además, algunas bases de datos cuentan con APIs (application programming interface) que cumplen la misma función de conectividad.

Vamos a aprender a trabajar con varias de estas herramientas con un ejemplo que desarrollaremos de a pasos.

## El metabolismo de la tirosina en *Saccharomyces cerevisiae*

### Trabajar con la base de datos KEGG

El metabolismo de la tirosina en *S. cerevisiae* es, de acuerdo a cómo lo define KEGG, una via con pocos pasos, lo que lo hace apropiado para nuestro ejemplo. Este es el link a esta vía en KEGG:

https://www.genome.jp/entry/pathway+sce00350

KEGG expone una API y tanto Biopython como Bioconductor cuentan con funciones para interactuar con esa API. El identificador KEGG del genoma de *S. cerevisiae* es "sce". A partir de este dato vamos a recopilar toda la información relevante sobre el metabolismo de la tirosina en *S. cerevisiae*. 

In [2]:
from Bio.KEGG import REST
vias_levadura = REST.kegg_list("pathway", "sce").read()

In [3]:
help(REST.kegg_list)

Help on function kegg_list in module Bio.KEGG.REST:

kegg_list(database, org=None)
    KEGG list - Entry list for database, or specified database entries.
    
    db - database or organism (string)
    org - optional organism (string), see below.
    
    For the pathway and module databases the optional organism can be
    used to restrict the results.



¿Qué información recuperamos?

In [4]:
type(vias_levadura)

str

In [5]:
len(vias_levadura)

9768

In [4]:
vias_levadura[0:300]

'path:sce00010\tGlycolysis / Gluconeogenesis - Saccharomyces cerevisiae (budding yeast)\npath:sce00020\tCitrate cycle (TCA cycle) - Saccharomyces cerevisiae (budding yeast)\npath:sce00030\tPentose phosphate pathway - Saccharomyces cerevisiae (budding yeast)\npath:sce00040\tPentose and glucuronate interconve'

Tenemos un string de 9768 caracteres. Dentro del string hay separadores de linea "\n". Cada línea parece corresponder a una via metabólica, donde primero aparece el número de acceso de la vía, después del identificador "path:", luego hay una marca de tabulación "\t" y sigue el nombre de la vía metabólica. Confirmemos:

In [6]:
for linea in vias_levadura.split("\n"):
    print(linea)


path:sce00010	Glycolysis / Gluconeogenesis - Saccharomyces cerevisiae (budding yeast)
path:sce00020	Citrate cycle (TCA cycle) - Saccharomyces cerevisiae (budding yeast)
path:sce00030	Pentose phosphate pathway - Saccharomyces cerevisiae (budding yeast)
path:sce00040	Pentose and glucuronate interconversions - Saccharomyces cerevisiae (budding yeast)
path:sce00051	Fructose and mannose metabolism - Saccharomyces cerevisiae (budding yeast)
path:sce00052	Galactose metabolism - Saccharomyces cerevisiae (budding yeast)
path:sce00053	Ascorbate and aldarate metabolism - Saccharomyces cerevisiae (budding yeast)
path:sce00061	Fatty acid biosynthesis - Saccharomyces cerevisiae (budding yeast)
path:sce00062	Fatty acid elongation - Saccharomyces cerevisiae (budding yeast)
path:sce00071	Fatty acid degradation - Saccharomyces cerevisiae (budding yeast)
path:sce00072	Synthesis and degradation of ketone bodies - Saccharomyces cerevisiae (budding yeast)
path:sce00100	Steroid biosynthesis - Saccharomyces c

Ahora deberíamos buscar línea por línea cuál es la vía que corresponde al metabolismo de la tirosina... ¡De ninguna manera! ¿Para qué estamos aprendiendo a programar? Necesitamos buscar el texto "tyrosine" en cada línea.

In [6]:
for linea in vias_levadura.split("\n"):
    if 'tyrosine' in linea:
        print(linea)


path:sce00400	Phenylalanine, tyrosine and tryptophan biosynthesis - Saccharomyces cerevisiae (budding yeast)


Mmmm... Pasó algo acá. KEGG cuenta con una vía específica del metabolismo de la tirosina, lo que recuperamos es una que se refiere sólo a la biosíntesis.

La respuesta tiene que ver con una consideración importante: las búsquedas de texto en Python (y en R, y en linux) son sensibles a mayúsculas y minúsculas. Esto es, nuestra búsqueda anterior no va a devolver cómo resultado una línea con el texto "Tyrosine". 

Una forma fácil de reolver esto es convertir el texto de la línea a minúsculas. Los strings tienen un método para esto:

In [7]:
for linea in vias_levadura.split("\n"):
    if 'tyrosine' in linea.lower():
        print(linea)

path:sce00350	Tyrosine metabolism - Saccharomyces cerevisiae (budding yeast)
path:sce00400	Phenylalanine, tyrosine and tryptophan biosynthesis - Saccharomyces cerevisiae (budding yeast)


¡Ahora sí! La vía que nos interesa es la que tiene número de acceso sce00350.

Vamos a buscar la información de esta vía en KEGG usando la función *REST.kegg_get()*. Esta función devuelve un gestor de archivos o *handle*. Esto implica que una vez que tenemos el *handle* de la via de interés podemos operar como si fuera un archivo:

In [44]:
via_tirosina = REST.kegg_get("sce00350")
via_tirosina_local = via_tirosina.read()

In [45]:
type(via_tirosina_local)

str

In [46]:
len(via_tirosina_local)

6316

In [11]:
via_tirosina_local[0:300]

'ENTRY       sce00350                    Pathway\nNAME        Tyrosine metabolism - Saccharomyces cerevisiae (budding yeast)\nCLASS       Metabolism; Amino acid metabolism\nPATHWAY_MAP sce00350  Tyrosine metabolism\nDBLINKS     GO: 0006570\nORGANISM    Saccharomyces cerevisiae (budding yeast) [GN:sce]\nGEN'

También podemos guardar el texto del registro en un archivo de texto en nuestro directorio de trabajo. Esto nos va a ser para entender el tipo de información y pensar cómo podemos continuar:

In [12]:
open("via_tirosina.txt", "w").write(via_tirosina_local)

6316

In [47]:
print(via_tirosina_local)

ENTRY       sce00350                    Pathway
NAME        Tyrosine metabolism - Saccharomyces cerevisiae (budding yeast)
CLASS       Metabolism; Amino acid metabolism
PATHWAY_MAP sce00350  Tyrosine metabolism
DBLINKS     GO: 0006570
ORGANISM    Saccharomyces cerevisiae (budding yeast) [GN:sce]
GENE        YLR027C  AAT2; aspartate transaminase AAT2 [KO:K14454] [EC:2.6.1.1]
            YKL106W  AAT1; aspartate transaminase AAT1 [KO:K14455] [EC:2.6.1.1]
            YIL116W  HIS5; histidinol-phosphate transaminase [KO:K00817] [EC:2.6.1.9]
            YGL202W  ARO8; bifunctional 2-aminoadipate transaminase/aromatic-amino-acid:2-oxoglutarate transaminase [KO:K00838] [EC:2.6.1.57 2.6.1.39 2.6.1.27 2.6.1.5]
            YHR137W  ARO9; aromatic-amino-acid:2-oxoglutarate transaminase [KO:K05821] [EC:2.6.1.58 2.6.1.28]
            YDL168W  SFA1; bifunctional alcohol dehydrogenase/S-(hydroxymethyl)glutathione dehydrogenase [KO:K00121] [EC:1.1.1.284 1.1.1.1]
            YOL086C  ADH1; alcohol dehy

Observen que en la línea anterior creamos el *handle* y a continuación llamamos el método para escribir la información de la vía metabólica. Todo en la misma línea.

¿Cómo seguimos? Vamos a parsear el texto y recuperar los nombres de los genes que participan de la vía metabólica. Si prestamos atención al texto de la vía metabólica vemos que hay secciones definidas por términos como "ENTRY", "CLASS" o "GENE", que son los términos que se ubican en los primeros 12 caracteres de algunas línea.

También observen en el texto del registro que una vez que se define una sección, por ejemplo "GENE", cada línea es un gen nuevo, pero no se repite el nombre de la sección: los primeros carácteres de las líneas siguientes quedan en blanco hasta llegar a la sección "COMPOUND".

Vamos a poner todo esto en uso. Primero recuperamos las lineas que corresponden a genes:

In [13]:
current_section = ""

for linea in via_tirosina_local.split("\n"):
    if "GENE" in linea[0:11]:
        current_section = linea[0:11].strip()
        print(linea)
    elif current_section == "GENE" and linea[0:11].strip() == "":    
        print(linea)
    else:
        current_section = "other"



GENE        YLR027C  AAT2; aspartate transaminase AAT2 [KO:K14454] [EC:2.6.1.1]
            YKL106W  AAT1; aspartate transaminase AAT1 [KO:K14455] [EC:2.6.1.1]
            YIL116W  HIS5; histidinol-phosphate transaminase [KO:K00817] [EC:2.6.1.9]
            YGL202W  ARO8; bifunctional 2-aminoadipate transaminase/aromatic-amino-acid:2-oxoglutarate transaminase [KO:K00838] [EC:2.6.1.57 2.6.1.39 2.6.1.27 2.6.1.5]
            YHR137W  ARO9; aromatic-amino-acid:2-oxoglutarate transaminase [KO:K05821] [EC:2.6.1.58 2.6.1.28]
            YDL168W  SFA1; bifunctional alcohol dehydrogenase/S-(hydroxymethyl)glutathione dehydrogenase [KO:K00121] [EC:1.1.1.284 1.1.1.1]
            YOL086C  ADH1; alcohol dehydrogenase ADH1 [KO:K13953] [EC:1.1.1.1]
            YMR303C  ADH2; alcohol dehydrogenase ADH2 [KO:K13953] [EC:1.1.1.1]
            YBR145W  ADH5; alcohol dehydrogenase ADH5 [KO:K13953] [EC:1.1.1.1]
            YMR083W  ADH3; alcohol dehydrogenase ADH3 [KO:K13953] [EC:1.1.1.1]
            YGL256W 

¿Cómo funciona esto? En cada iteración preguntamos si ahí comienza la sección "GENE". Si esto es cierto, actualizamos el string *current_section* (pregunta ¿qué hace el método *strip()*?). 

Si esto es falso, preguntamos si las primeros caracteres de la línea están vacios y si *current_section* sigue siendo "GENE". Las líneas que cumplen con esto también nos interesan. Finalmente, si nada de esto se cumple, es porque el iterador ya entró en otra sección.

Vamos a agregar algunas cosas más:

In [50]:
gene_ids = list()
current_section = ""

for linea in via_tirosina_local.split("\n"):
    if "GENE" in linea[0:11]:
        current_section = linea[0:11].strip()
        gene, gene_description = linea[12:].split("; ")
        gene_id, gene_symbol = gene.split()
        gene_ids.append(gene_id)
    elif current_section == "GENE" and linea[0:11].strip() == "":
        gene, gene_description = linea[12:].split("; ")
        gene_id, gene_symbol = gene.split()
        gene_ids.append(gene_id)
    else:
        current_section = "other"


En esta nueva variante del script generamos una lista con los números de acceso de los genes involucrados en nuestra via de interés. Vamos a usar el primer elemento de esa lista para recuperar su registro correspondiente.

In [15]:
gene_ids[0]

'YLR027C'

Para recuperar el registro, especificamos que es un gen de *S.carevisiae* con el prefijo "sce:".
También podemos ejecutar *read()* sobre el *handle* de nuestra copia local.

In [16]:
primer_gen_local = REST.kegg_get("sce:" + gene_ids[0]).read()

In [52]:
len(primer_gen_local)

5239

In [18]:
primer_gen_local[0:200]

'ENTRY       YLR027C           CDS       T00005\nNAME        AAT2, ASP5\nDEFINITION  (RefSeq) aspartate transaminase AAT2\nORTHOLOGY   K14454  aspartate aminotransferase, cytoplasmic [EC:2.6.1.1]\nORGANISM'

Como hicimos antes, podemos guardar un archivo en nuestra carpeta de trabajo:

In [19]:
open("sce_YLR027C.txt", "w").write(primer_gen_local)

5239

Al mirar este archivo vemos que hacia el final se encuentra esta información:

<pre>
DBLINKS     NCBI-GeneID: 850714
            NCBI-ProteinID: NP_013127
            SGD: S000004017
            UniProt: P23542
</pre>

### Propuesta de ejercicio

La información que recién vimos sobre la sección DBLINKS en el registro del gen YLR027C se puede recuperar programáticamente haciendo una pequeñas modificaciones al código que vimos para el análisis de la vía metabólica. Intenten hacerlo.

Opcional: Hay dos funciones *REST.kegg_info()* y *REST.kegg_find()*. No son de uso tan frecuente como las que acabamos de ver, pero pueden buscar algo de información sobre ellas. Aqui tienen dos ejemplos para empezar a probrar qué hacen:

<pre>
sce_info = REST.kegg_info("sce").read()
print(sce_info)

info_tirosina = REST.kegg_find("sce", "tyrosin").read()
print(info_tirosina)
</pre>

### Recuperar información desde el NCBI

El NCBI expone la API pública "Entrez Programming Utilities (E-utilities)" y Biopython tiene herramientas para interactuar con esa API. En general, las clases o funciones que permiten interactuar con una API pública se conocen con el nombre de *wrappers*.

Vamos a trabajar con estas funciones.

In [20]:
from Bio import Entrez
Entrez.email = "tu_mail@institucion.com.ar" 
# No es obligatorio, pero por cortesía está bueno agregar nuestro e-mail al usar la API del NCBI
handle = Entrez.einfo()
bases_ncbi = Entrez.read(handle, validate = False)
handle.close()

¿Qué tipo de información recuperamos?

In [21]:
type(bases_ncbi)

Bio.Entrez.Parser.DictionaryElement

In [22]:
bases_ncbi.keys()

dict_keys(['DbList'])

¿Qué es esto?
La API del NCBI nos devolvió una lista de todas sus bases de datos en formato XML, que es un formato bastante popular para intercambiar información entre computadoras. Con la función *Entrez.read()* convertimos la información en XML directamente a una variante de diccionario, que es más fácil de trabajar. Además en algunos casos, el XML no aparece correctamente formateado.

In [23]:
bases_ncbi['DbList']

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']

Esta es la lista de bases de datos del NCBI y que están disponibles para consultar usando utilidades de la API.

En el último paso de la búsqueda por KEGG habíamos encontrado una referencia al registro del gen correspondiente en el NCBI: "NCBI-GeneID: 850714". Hay diferentes formas de consultar acerca un gen usando Entrez. Vamos por pasos:

In [24]:
summ_handle = Entrez.esummary(db="gene", id="850714")
summ_gene = Entrez.read(summ_handle)

Entrez nos devuelve una estrucura algo compleja, es una variante de diccionario que contiene otro diccionario, y que finalmente tiene una lista.

In [25]:
type(summ_gene)

Bio.Entrez.Parser.DictionaryElement

Ustedes pueden investigar los pasos intermedios, pero para abreviar, la información que nos interesa está aquí:

In [26]:
record_summ = summ_gene['DocumentSummarySet']['DocumentSummary'][0]

In [27]:
record_summ.keys()

dict_keys(['Name', 'Description', 'Status', 'CurrentID', 'Chromosome', 'GeneticSource', 'MapLocation', 'OtherAliases', 'OtherDesignations', 'NomenclatureSymbol', 'NomenclatureName', 'NomenclatureStatus', 'Mim', 'GenomicInfo', 'GeneWeight', 'Summary', 'ChrSort', 'ChrStart', 'Organism', 'LocationHist'])

In [28]:
for key in record_summ:
    print(key, ':', record_summ[key])

Name : AAT2
Description : aspartate transaminase AAT2
Status : 0
CurrentID : 0
Chromosome : XII
GeneticSource : genomic
MapLocation : 
OtherAliases : YLR027C, ASP5
OtherDesignations : aspartate transaminase AAT2
NomenclatureSymbol : 
NomenclatureName : 
NomenclatureStatus : 
Mim : []
GenomicInfo : [{'ChrLoc': 'XII', 'ChrAccVer': 'NC_001144.5', 'ChrStart': '198084', 'ChrStop': '196828', 'ExonCount': '1'}]
GeneWeight : 1753
Summary : 
ChrSort : 12
ChrStart : 196828
Organism : {'ScientificName': 'Saccharomyces cerevisiae S288C', 'CommonName': '', 'TaxID': '559292'}
LocationHist : []


Si quisiéramos recuperar la información en formato texto, como se muestra en la página web del NCBI, tenemos que proceder de otra forma, usando la función *Entrez.efetch()*:

In [29]:
full_handle = Entrez.efetch(db="gene", id="850714", rettype="gb", retmode="text")
full_gene = full_handle.read()

Observen que en la segunda linea usamos una variante de *Entrez.read()*, pero el principio es el mismo:

1. Comunicarse con el NCBi via Entrez y crear un *handle*
2. Convertir el *handle* a algún objeto con el que podramos trabajar

In [30]:
print(full_gene)


1. AAT2
aspartate transaminase AAT2 [Saccharomyces cerevisiae S288C]
Other Aliases: YLR027C, ASP5
Other Designations: aspartate transaminase AAT2
Chromosome: XII
Annotation: Chromosome XII NC_001144.5 (196829..198085, complement)
ID: 850714




### Propuesta de ejercicio

Además del identificador del gen, el registro de KEGG incluía una referencia a la base de datos de proteínas del NCBI, "NCBI-ProteinID: NP_013127", pueden repetir el proceso anterior para recuperar este registro. 

### Recuperar información desde SGD (Saccharomyces Genome Database)

La información inicial que teníamos de este gen incluía un registro en SGD: "SGD: S000004017". Biopython no tiene un *wrapper* de la API de SGD, pero podemos acceder a la API de SGD usando una tecnología llamada cURL o a través de una URL. 

cURL es un projecto que creó y mantiene una biblioteca o *library* llamada *libcurl* y una herramienta para usar por línea de comando (curl) para transferir datos mediante diferentes procotocols de red. Este sería el método más general y el que soportan la mayoría de las APIs. En nuestro caso, al contar con la posibilidad de interrogar la API con una URL nos permite escribir código más limpio, y con el mismo resultado


In [31]:
import requests
sgd_record = requests.get('https://www.yeastgenome.org/backend/locus/S000004017')

La información que tenemos en *sgd_record* está en un formato que se llama JSON ("jason"), que es otro formato popular para intercambo de información. También se puede ver como un string en formato texto, que es dificil de leer. Las llamadas son:

<pre>
sgd_record.text
sgd_record.json()
</pre>

Vamos a parsear el registro en formato JSON. Primero veamos el tipo:

In [32]:
type(sgd_record.json())

dict

In [33]:
type(sgd_record.json())

dict

Es un diccionario, así que podemos buscar las claves:

In [34]:
sgd_record.json().keys()

dict_keys(['complexes', 'urls', 'interaction_overview', 'sgdid', 'paralogs', 'bioent_status', 'ecnumbers', 'locus_type', 'main_strain', 'history', 'disease_overview', 'literature_overview', 'format_name', 'qualifier', 'reference_mapping', 'paragraph', 'id', 'regulation_overview', 'description', 'link', 'genetic_position', 'phenotype_overview', 'qualities', 'aliases', 'complements', 'protein_overview', 'references', 'pathways', 'go_overview', 'gene_name', 'name_description', 'display_name'])

A continuación siguen unos ejemplos:
El primero es un valor de tipo string.

In [35]:
sgd_record.json()["main_strain"]

'S288C'

En este otro caso el valor es un diccionario.

In [36]:
sgd_record.json()["literature_overview"]

{'total_count': 111,
 'additional_count': 44,
 'review_count': 8,
 'primary_count': 21}

Y en este otro son varios diccionarios anidados.

In [37]:
sgd_record.json()["go_overview"]

{'paragraph': 'L-aspartate:2-oxoglutarate aminotransferase that binds pyridoxal phosphate; involved in amino acid metabolism ; localizes to mitochondria, peroxisome and cytsol',
 'htp_biological_process_terms': [],
 'manual_cellular_component_terms': [{'namespace': 'cellular component',
   'term': {'display_name': 'cytosol', 'link': '/go/GO:0005829'},
   'qualifiers': ['part of'],
   'evidence_codes': [{'display_name': 'IDA',
     'link': 'http://wiki.geneontology.org/index.php/Inferred_from_Direct_Assay_(IDA)'}]},
  {'namespace': 'cellular component',
   'term': {'display_name': 'peroxisome', 'link': '/go/GO:0005777'},
   'qualifiers': ['part of'],
   'evidence_codes': [{'display_name': 'IDA',
     'link': 'http://wiki.geneontology.org/index.php/Inferred_from_Direct_Assay_(IDA)'},
    {'display_name': 'IMP',
     'link': 'http://wiki.geneontology.org/index.php/Inferred_from_Mutant_Phenotype_(IMP)'}]}],
 'htp_cellular_component_terms': [],
 'manual_molecular_function_terms': [{'namespa

Para algunos campos complejos, como el anterior, la API de SGD permite interrogar ese item en forma específica. Para más información pueden consultar:

https://www.yeastgenome.org/api/doc#/

Dependiendo de la información que necesitemos recuperar de SGD es posible acceder a ella en forma programática con mayor o menor dificultad, para lograrlo es importante conocer cómo es la estructura de los datos con los que vamos a trabajar. Esta es una de las desventajas de no contar con un *wrapper*, pero lo esencial es que la información con más o menos trabajo, se puede extraer.
